In [ ]:
import numpy as np
from math import sqrt
import random
import heapq

class Event:
    '''
    Store the properties of one event in the Schedule class defined below. Each
    event has a time at which it needs to run, a function to call when running
    the event, along with the arguments and keyword arguments to pass to that
    function.
    '''
    def __init__(self, timestamp, function, *args, **kwargs):
        self.timestamp = timestamp
        self.function = function
        self.args = args
        self.kwargs = kwargs

    def __lt__(self, other):
        '''
        This overloads the less-than operator in Python. We need it so the
        priority queue knows how to compare two events. We want events with
        earlier (smaller) times to go first.
        '''
        return self.timestamp < other.timestamp

    def run(self, schedule):
        '''
        Run an event by calling the function with its arguments and keyword
        arguments. The first argument to any event function is always the
        schedule in which events are being tracked. The schedule object can be
        used to add new events to the priority queue.
        '''
        self.function(schedule, *self.args, **self.kwargs)


class Schedule:
    '''
    Implement an event schedule using a priority queue. You can add events and
    run the next event.
    
    The `now` attribute contains the time at which the last event was run.
    '''
    
    def __init__(self):
        self.now = 0  # Keep track of the current simulation time
        self.priority_queue = []  # The priority queue of events to run
    
    def add_event_at(self, timestamp, function, *args, **kwargs):
        # Add an event to the schedule at a particular point in time.
        heapq.heappush(
            self.priority_queue,
            Event(timestamp, function, *args, **kwargs))
    
    def add_event_after(self, interval, function, *args, **kwargs):
        # Add an event to the schedule after a specified time interval.
        self.add_event_at(self.now + interval, function, *args, **kwargs)
    
    def next_event_time(self):
        # Return the time of the next event. The `now` attribute of this class
        # contain the time of the last event that was run.
        return self.priority_queue[0].timestamp

    def run_next_event(self):
        # Get the next event from the priority queue and run it.
        event = heapq.heappop(self.priority_queue)
        self.now = event.timestamp
        event.run(self)
        
    def __repr__(self):
        return (
            f'Schedule() at time {self.now} ' +
            f'with {len(self.priority_queue)} events in the queue')
    
    def print_events(self):
        # Print out diagnostic information about the events in the schedule.
        print(repr(self))
        for event in sorted(self.priority_queue):
            print(f'   {event.timestamp}: {event.function.__name__}')

In [ ]:
class Queue:
    def __init__(self, service_distribution):
        self.service_distribution = service_distribution
        # We start with an empty queue and the server not busy
        self.people_in_queue = 0
        self.people_being_served = 0

    def add_customer(self, schedule):
        # Add the customer to the queue
        self.people_in_queue += 1
        if self.people_being_served < 1:
            # This customer can be served immediately
            schedule.add_event_after(0, self.start_serving_customer)
            
    def start_serving_customer(self, schedule):
        # from queue to a server
        self.people_in_queue -= 1
        self.people_being_served += 1
        schedule.add_event_after(
            self.service_distribution.rvs(),
            self.finish_serving_customer)
            
    def finish_serving_customer(self, schedule):
        # Remove the customer
        self.people_being_served -= 1
        if self.people_in_queue > 0:
            # serve the next customer
            schedule.add_event_after(0, self.start_serving_customer)


class BusSystem:
    
    def __init__(self, arrival_distribution, service_distribution):
        self.queue = Queue(service_distribution)
        self.arrival_distribution = arrival_distribution

    def add_customer(self, schedule):
        # Add this customer to the queue
        self.queue.add_customer(schedule)
        # Schedule when to add another customer
        schedule.add_event_after(
            self.arrival_distribution.rvs(),
            self.add_customer)

    def run(self, schedule):
        # Schedule when the first customer arrives
        schedule.add_event_after(
            self.arrival_distribution.rvs(),
            self.add_customer)

In [ ]:
def run_experiment(arrival_rate, departure_rate, final_time):
    '''
    Run the queueing experiment for particular arrival and departure ratews.
    This will make a plot of the number of people in the queue over time.
    
    Inputs:
    
        arrival_rate (float) The rate at which new people join the back of the
          queue. Units: people per second.
        
        departure_rate (float) The rate at which people are served and leave
          the queue. Units people per second.
        
        final_time (float) Until what time to run the simulation. When the next
        arrival and departure times are later than this, the simulation will
        end. Units: seconds.

    Returns: None
    '''

    # Set up the exponential distributions for arrivals and departures
    arrival_distribution = sts.expon(scale=1/arrival_rate)
    departure_distribution = sts.expon(scale=1/departure_rate)

    people_in_queue = 0  # Start with an empty queue
    next_arrival_time = arrival_distribution.rvs()  # Generate the first arrival
    
    # We need at least 1 arrival before we can have departures so we set the
    # first departure time to infinity. This will be modified below.
    next_departure_time = np.inf  

    plot_data = {
        'time': [],
        'people_in_queue': []}
    
    ### I cant quite figure out what to do here
        
    # Make the plot of the queue length over time
    plt.figure()
    plt.title('Simulated queue length over time')
    plt.plot(plot_data['time'], plot_data['people_in_queue'], 'k.')
    plt.xlabel('time [s]')
    plt.ylabel('queue length')
    plt.show()

AttributeError: ignored

I applied #pythonimplementation by using OOP to create the queue and busSystem class. I believe that this is the first step of solving the overall problem as I will require some of the assumptions such as the total capacity and travel limit to be initialized globally, and then certain more classes to be incorporated into the programming that I have done. Unfortunately, I could not fully test these two classes since I feel there are still missing pieces to implementation of M/D/1. Nevertheless, I think by commenting to describe what I have been doing, I have done a solid enough job on #codereadability as well, which will help me later on.